You can get the dataset used in this source code from:
https://www.kaggle.com/joaopauloschuler/cifar10-128x128-resized-via-cai-super-resolution .

You might need to install this on your system:
apt-get install python3-opencv git

In [ ]:
import os

if not os.path.isdir('k'):
 !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull

!cd k && pip install .

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.densenet
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import mixed_precision
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
from sklearn.metrics import classification_report

Tensorflow version: 2.4.1
Keras version: 2.4.0


In [ ]:
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [ ]:
verbose=True
lab=False
bipolar=True
base_model_name='JP30B04'

In [ ]:
num_classes = 10
batch_size = 64
epochs = 50
target_size_x = 224
target_size_y = 224
seed = 12

In [ ]:
root_folder = '' # Change this as required
data_dir = root_folder + 'cifar10-128/train';
test_dir = root_folder + 'cifar10-128/test';

In [ ]:
train_datagen = cai.util.create_image_generator(validation_split=0.1)
test_datagen = cai.util.create_image_generator_no_augmentation()
cpus_num = max([multiprocessing.cpu_count(), 32])

def lrscheduler(epoch):
  return 0.003 * (0.99**epoch)

In [ ]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 2, 1]:
      basefilename = 'DEV-EfficientNet-CIFAR10-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.efficientnet.EfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes)
      else:
        model = cai.efficientnet.kEfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes,
          kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): model.summary()

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        train_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="training",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        validation_flow = train_datagen.flow_from_directory(
            directory=data_dir,
            subset="validation",
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        history = model.fit(
          x = train_flow,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=validation_flow,
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(lrscheduler)],
          workers=cpus_num,
          max_queue_size=128
          )
      if (test_results):
        test_flow = test_datagen.flow_from_directory(
            directory=test_dir,
            target_size=(target_size_x, target_size_y),
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="categorical",
            shuffle=True,
            seed=seed
        )
        print('Best Model Results: '+best_result_file_name)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(
            x=test_flow,
            batch_size=batch_size,
            use_multiprocessing=False,
            workers=cpus_num
        )
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(x_test)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(y_test, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False)

Running: DEV-EfficientNet-CIFAR10--1
Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
_______________________________________________

# Fitting

In [ ]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=True)

Running: DEV-EfficientNet-CIFAR10--1
Found 45000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.
Epoch 1/50
704/704 [==============================] - 276s 356ms/step - loss: 2.5936 - accuracy: 0.2294 - val_loss: 3.9349 - val_accuracy: 0.2806

Epoch 00001: val_accuracy improved from -inf to 0.28060, saving model to DEV-EfficientNet-CIFAR10--1-best_result.hdf5
Epoch 2/50
704/704 [==============================] - 250s 347ms/step - loss: 1.5130 - accuracy: 0.4553 - val_loss: 1.4837 - val_accuracy: 0.4968

Epoch 00002: val_accuracy improved from 0.28060 to 0.49680, saving model to DEV-EfficientNet-CIFAR10--1-best_result.hdf5
Epoch 3/50
704/704 [==============================] - 254s 347ms/step - loss: 1.1531 - accuracy: 0.5938 - val_loss: 1.1547 - val_accuracy: 0.6032

Epoch 00003: val_accuracy improved from 0.49680 to 0.60320, saving model to DEV-EfficientNet-CIFAR10--1-best_result.hdf5
Epoch 4/50
704/704 [==============================] - 254s 348ms/step - los

# Test Results

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True)

Running: DEV-EfficientNet-CIFAR10--1
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-EfficientNet-CIFAR10--1-best_result.hdf5
157/157 [==============================] - 21s 118ms/step - loss: 0.2676 - accuracy: 0.9166
loss 0.2676483392715454
acc 0.9165999889373779
Finished: DEV-EfficientNet-CIFAR10--1
Running: DEV-EfficientNet-CIFAR10-2
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-EfficientNet-CIFAR10-2-best_result.hdf5
157/157 [==============================] - 41s 181ms/step - loss: 0.2708 - accuracy: 0.9164
loss 0.2707709074020386
acc 0.9164000153541565
Finished: DEV-EfficientNet-CIFAR10-2
Running: DEV-EfficientNet-CIFAR10-1
Found 10000 images belonging to 10 classes.
Best Model Results: DEV-EfficientNet-CIFAR10-1-best_result.hdf5
157/157 [==============================] - 38s 174ms/step - loss: 0.2770 - accuracy: 0.9159
loss 0.2769763767719269
acc 0.9158999919891357
Finished: DEV-EfficientNet-CIFAR10-1
